In [ ]:
# imports
from openai import OpenAI
import gradio as gr

In [ ]:
ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
models = ["gpt-oss:latest", "qwen3-coder:latest",]
clients = {"gpt-oss:latest": ollama, "qwen3-coder:latest": ollama,}

In [ ]:
def system_prompt_for_docstring(language):
    """
    Generate system prompt for docstring generation based on programming language.
    
    Args:
        language (str): Programming language (python, javascript, java, etc.)
    
    Returns:
        str: System prompt tailored for the specified language
    """
    prompts = {
        "python": """
        You are a Python documentation expert. When writing documentation:
        - Follow PEP 257 and Google docstring style guidelines
        - Write clear, concise explanations
        - Include practical examples when helpful
        - Highlight edge cases and limitations
        - Use type hints in docstrings
        - Add inline comments only for complex logic
        - Never skip documenting parameters or return values
        - Validate that all documentation is accurate and complete
        """,
        "javascript": """
        You are a JavaScript/TypeScript documentation expert. When writing documentation:
        - Follow JSDoc standards
        - Write clear, concise explanations
        - Include type annotations
        - Document parameters, return values, and exceptions
        - Add inline comments for complex logic
        - Use modern ES6+ syntax examples
        """,
        "java": """
        You are a Java documentation expert. When writing documentation:
        - Follow Javadoc standards
        - Write clear, concise explanations
        - Document all public methods and classes
        - Include @param, @return, and @throws tags
        - Add inline comments for complex logic
        """,
        "cpp": """
        You are a C++ documentation expert. When writing documentation:
        - Follow Doxygen standards
        - Write clear, concise explanations
        - Document parameters, return values, and exceptions
        - Add inline comments for complex logic and memory management
        """,
        "go": """
        You are a Go documentation expert. When writing documentation:
        - Follow Go documentation conventions
        - Write clear, concise explanations
        - Document exported functions and types
        - Add inline comments for complex logic
        """,
        "rust": """
        You are a Rust documentation expert. When writing documentation:
        - Follow Rust documentation conventions
        - Write clear, concise explanations
        - Document safety considerations
        - Include examples in doc comments
        """,
    }
    return prompts.get(language.lower(), prompts["python"])

In [ ]:
def user_prompt_for_docstring(code, language):
    """
    Generate user prompt for docstring generation request.
    
    Args:
        code (str): Source code to document
        language (str): Programming language of the code
    
    Returns:
        str: Formatted user prompt
    """
    return f"""
    Please document this {language} code with comprehensive docstrings and comments:
    
    1. Add docstrings containing:
       - Clear description of purpose and functionality
       - All parameters with types and descriptions
       - Return values with types
       - Exceptions that may be raised
       - Any important notes or limitations
    
    2. Add strategic inline comments for:
       - Complex algorithms or business logic
       - Non-obvious implementation choices
       - Performance considerations
       - Edge cases
    
    Return ONLY the documented code, no explanations before or after.
    
    Here's the code to document:
    
    {code}
    """

In [ ]:
def stream_docstring_ollama(code, language, model):
    """
    Generate docstrings using open-source models with streaming via Ollama.
    
    Args:
        code (str): Source code to document
        language (str): Programming language
    
    Yields:
        str: Progressively generated documented code
    """
    messages = [
        {"role": "system", "content": system_prompt_for_docstring(language)},
        {"role": "user", "content": user_prompt_for_docstring(code, language)}
    ]
    
    stream = ollama.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
        temperature=0.3
    )
    
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python', '').replace('```javascript', '').replace('```java', '').replace('```', '')

In [ ]:
def generate_docstring(code, language, model):
    """
    Main function to generate docstrings using selected AI model.
    
    Args:
        code (str): Source code to document
        language (str): Programming language
        model (str): AI model to use (GPT, Claude, or Gemini)
    
    Yields:
        str: Progressively generated documented code
    
    Raises:
        ValueError: If unknown model is specified
    """
    if not code.strip():
        yield "Please enter some code to document."
        return
    
    result = stream_docstring_ollama(code, language, model)

    for stream_so_far in result:
        yield stream_so_far

In [ ]:
# Example code for testing
EXAMPLE_PYTHON_CODE = """
def calculate_pi(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

class DataProcessor:
    def __init__(self, data):
        self.data = data
        self.processed = False
    
    def process(self, threshold=0.5):
        if not self.data:
            raise ValueError("No data to process")
        result = [x for x in self.data if x > threshold]
        self.processed = True
        return result
"""

EXAMPLE_JAVASCRIPT_CODE = """
function calculateSum(numbers) {
    return numbers.reduce((acc, num) => acc + num, 0);
}

class UserManager {
    constructor(users) {
        this.users = users;
    }
    
    findByAge(minAge, maxAge) {
        return this.users.filter(user => 
            user.age >= minAge && user.age <= maxAge
        );
    }
}
"""

EXAMPLE_JAVA_CODE = """
public class Calculator {
    private double result;
    
    public Calculator() {
        this.result = 0.0;
    }
    
    public double add(double a, double b) {
        result = a + b;
        return result;
    }
    
    public double divide(double a, double b) {
        if (b == 0) {
            throw new ArithmeticException("Division by zero");
        }
        result = a / b;
        return result;
    }
}
"""

In [ ]:
# Custom CSS for better UI
css = """
.code-input textarea, .code-output textarea {
    font-family: 'Courier New', monospace;
    font-size: 14px;
}
.header {
    text-align: center;
    padding: 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 10px;
    margin-bottom: 20px;
}
"""

# Create Gradio interface
with gr.Blocks(css=css, theme=gr.themes.Soft()) as ui:
    gr.Markdown("""
    <div class="header">
        <h1>🚀 AI Docstring Generator</h1>
        <p>Automatically generate comprehensive docstrings and comments for your code</p>
    </div>
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Configuration")
            language_dropdown = gr.Dropdown(
                choices=["Python", "JavaScript", "Java", "C++", "Go", "Rust"],
                label="Programming Language",
                value="Python"
            )
            model_dropdown = gr.Dropdown(
                choices=models,
                label="Select model",
                value=models[0],
                info="Select which LLM model to use"
            )
            
            gr.Markdown("### 📝 Examples")
            example_dropdown = gr.Dropdown(
                choices=["Python Example", "JavaScript Example", "Java Example", "Custom"],
                label="Load Example",
                value="Python Example"
            )
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📥 Input Code")
            code_input = gr.Textbox(
                label="Paste your code here",
                value=EXAMPLE_PYTHON_CODE,
                lines=20,
                placeholder="Enter your code...",
                elem_classes="code-input"
            )
            generate_btn = gr.Button("✨ Generate Docstrings", variant="primary", size="lg")
        
        with gr.Column(scale=1):
            gr.Markdown("### 📤 Documented Code")
            code_output = gr.Textbox(
                label="Generated code with docstrings",
                lines=20,
                elem_classes="code-output"
            )
    
    gr.Markdown("""
    ### 📚 Instructions:
    1. Select your programming language
    2. Choose a downloaded open-source model in Ollama
    3. Paste your code or select an example
    4. Click "Generate Docstrings"
    5. Copy the documented code
    
    **Note:** Make sure to download the LLMs via CLI "ollama pull model_name" first!
    """)
    
    # Event handlers
    def load_example(example_name):
        examples = {
            "Python Example": EXAMPLE_PYTHON_CODE,
            "JavaScript Example": EXAMPLE_JAVASCRIPT_CODE,
            "Java Example": EXAMPLE_JAVA_CODE,
            "Custom": ""
        }
        return examples.get(example_name, "")
    
    example_dropdown.change(
        fn=load_example,
        inputs=[example_dropdown],
        outputs=[code_input]
    )
    
    generate_btn.click(
        fn=generate_docstring,
        inputs=[code_input, language_dropdown, model_dropdown],
        outputs=[code_output]
    )

In [ ]:
# Launch the interface

ui.launch(
    inbrowser=True,
    share=False
)